### Convert: Google Earth Assets to 56 CSV Files 

* National average embeddings data per county for all states (2017 to 2024).
* Each asset represents one state (according to the FIPS code).
* State FIPS Codes available here: https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt

Using the `convert_to_df()` function from `utils.py`: 

In [ ]:
from utils import convert_to_df